In [1]:
import os

import sys  # <- This is necessary!

# os.chdir('C:\\Users\\Administrator\\Downloads\\ML Lab')
#sys.path.append(r'D:\anaconda\envs')
sys.path.append(r'D:\anaconda\envs')

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback

In [3]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [4]:
def MLP():
    model = Sequential()
    model.add(Flatten(input_shape=(time_steps , num_features)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    return model

In [5]:
model1 = MLP()
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 504)               0         
                                                                 
 dense (Dense)               (None, 32)                16160     
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16,193
Trainable params: 16,193
Non-trainable params: 0
_________________________________________________________________


In [6]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [7]:
checkpoints = r'D:\anaconda\envs\lab7\checkpoint\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'D:\anaconda\envs\lab7\checkpoint'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [8]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [9]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =MLP()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [10]:
import os
path_dataset =r'D:\anaconda\envs\lab7'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

D:\anaconda\envs\MLLAB\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [11]:
df_te.head(1)

,aep,Is_holiday1,Is_holiday2,Is_Weekend1,Is_Weekend2,sin_month,cos_month,sin_week,cos_week,sin_hour,...,sin_wintert,cos_wintert,sin_springt,cos_springt,sin_summert,cos_summert,sin_fallt,cos_fallt,sin_year_dayt,cos_year_dayt
0,0.518532,1.0,0.0,1.0,0.0,1.0,6.123234e-17,0.866025,-0.5,-0.707107,...,0.0,1.0,1.0,6.123234e-17,0.0,1.0,0.0,1.0,0.956235,0.2926


In [12]:
test_set[0]

array([ 5.18532385e-01,  1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  6.12323400e-17,  8.66025404e-01,
       -5.00000000e-01, -7.07106781e-01, -7.07106781e-01,  0.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  6.12323400e-17,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  9.56234827e-01,
        2.92600336e-01])

In [13]:
time_steps=2
num_features=21

In [14]:
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=0)

In [15]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02]])

In [16]:
train_y[0]

array([], dtype=float64)

In [17]:
time_steps =24
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=1,target_len=2)

In [18]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.26232372e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e

In [19]:
train_y[0]

array([1., 1.])

In [20]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.45728468894958496 sec


In [21]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                    verbose = verbose)

Epoch 1/60
2637/2653 [============================>.] - ETA: 0s - loss: 0.0501 - mae: 0.0501 - mape: 821.6181
Epoch 1: val_loss improved from inf to 0.02639, saving model to D:\anaconda\envs\lab7\checkpoint\E1-cp-0001-loss0.03.h5
2653/2653 [==============================] - 8s 3ms/step - loss: 0.0500 - mae: 0.0500 - mape: 816.8565 - val_loss: 0.0264 - val_mae: 0.0264 - val_mape: 12.3231
Epoch 2/60
2640/2653 [============================>.] - ETA: 0s - loss: 0.0254 - mae: 0.0254 - mape: 17.9259
Epoch 2: val_loss improved from 0.02639 to 0.02076, saving model to D:\anaconda\envs\lab7\checkpoint\E1-cp-0002-loss0.02.h5
2653/2653 [==============================] - 6s 2ms/step - loss: 0.0254 - mae: 0.0254 - mape: 17.8778 - val_loss: 0.0208 - val_mae: 0.0208 - val_mape: 8.8302
Epoch 3/60
2642/2653 [============================>.] - ETA: 0s - loss: 0.0218 - mae: 0.0218 - mape: 383.7761
Epoch 3: val_loss did not improve from 0.02076
2653/2653 [==============================] - 7s 3ms/step - los

Epoch 36/60
83776/84882 [============================>.] - ETA: 0s - loss: 0.0120 - mean_absolute_error: 0.0120 - mean_absolute_percentage_error: 190.6300
Epoch 00036: val_loss did not improve from 0.01125
84882/84882 [==============================] - 2s 27us/sample - loss: 0.0120 - mean_absolute_error: 0.0120 - mean_absolute_percentage_error: 188.1922 - val_loss: 0.0117 - val_mean_absolute_error: 0.0117 - val_mean_absolute_percentage_error: 5.4416
Epoch 37/60
83936/84882 [============================>.] - ETA: 0s - loss: 0.0119 - mean_absolute_error: 0.0119 - mean_absolute_percentage_error: 142.9241
Epoch 00037: val_loss improved from 0.01125 to 0.01067, saving model to D:\anaconda\envs\lab7\checkpoint\\E1-cp-0037-loss0.01.h5
84882/84882 [==============================] - 2s 27us/sample - loss: 0.0119 - mean_absolute_error: 0.0119 - mean_absolute_percentage_error: 141.3728 - val_loss: 0.0107 - val_mean_absolute_error: 0.0107 - val_mean_absolute_percentage_error: 4.9973
Epoch 38/60
83

Epoch 54/60
84672/84882 [============================>.] - ETA: 0s - loss: 0.0111 - mean_absolute_error: 0.0111 - mean_absolute_percentage_error: 211.7884
Epoch 00054: val_loss did not improve from 0.00999
84882/84882 [==============================] - 2s 27us/sample - loss: 0.0111 - mean_absolute_error: 0.0111 - mean_absolute_percentage_error: 211.2738 - val_loss: 0.0119 - val_mean_absolute_error: 0.0119 - val_mean_absolute_percentage_error: 6.5517
Epoch 55/60
83328/84882 [============================>.] - ETA: 0s - loss: 0.0112 - mean_absolute_error: 0.0112 - mean_absolute_percentage_error: 187.9015
Epoch 00055: val_loss improved from 0.00999 to 0.00965, saving model to D:\anaconda\envs\lab7\checkpoint\\E1-cp-0055-loss0.01.h5
84882/84882 [==============================] - 2s 27us/sample - loss: 0.0112 - mean_absolute_error: 0.0112 - mean_absolute_percentage_error: 184.5262 - val_loss: 0.0096 - val_mean_absolute_error: 0.0096 - val_mean_absolute_percentage_error: 5.0974
Epoch 56/60
84

In [22]:
import h5py
f = h5py.File(r'D:\anaconda\envs\lab7\checkpoint\E1-cp-0055-loss0.01.h5', 'r')
print(list(f.keys()))

['model_weights', 'optimizer_weights']


In [23]:

model = load_model(r'D:\anaconda\envs\lab7\checkpoint\\E1-cp-0055-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 2ms/step
Mean Absolute Error (MAE): 149.2
Median Absolute Error (MedAE): 121.73
Mean Squared Error (MSE): 37490.87
Root Mean Squared Error (RMSE): 193.63
Mean Absolute Percentage Error (MAPE): 1.02 %
Median Absolute Percentage Error (MDAPE): 0.84 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [25]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the model (corrected path and loading method)
try:
    model = load_model(r'D:\anaconda\envs\lab7\checkpoint\E1-cp-0059-loss0.01.h5')
except Exception as e:
    print(f"Error loading model: {e}")
    # If the above fails, try this alternative:
    import h5py
    with h5py.File(r'D:\anaconda\envs\lab7\checkpoint\E1-cp-0059-loss0.01.h5', 'r') as f:
        model = load_model(f)

# Make predictions
y_pred_scaled = model.predict(test_X)

# Inverse transform the scaled data
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)

# Calculate metrics
def calculate_metrics(y_true, y_pred):
    errors = y_pred - y_true
    
    # Mean Absolute Error (MAE)
    MAE = np.mean(np.abs(errors))
    
    # Median Absolute Error (MedAE)
    MEDAE = np.median(np.abs(errors))
    
    # Mean Squared Error (MSE)
    MSE = np.mean(errors**2)
    
    # Root Mean Squared Error (RMSE)
    RMSE = np.sqrt(MSE)
    
    # Mean Absolute Percentage Error (MAPE)
    # Adding small epsilon to avoid division by zero
    epsilon = 1e-10
    MAPE = np.mean(np.abs(errors) / (np.abs(y_true) + epsilon)) * 100
    
    # Median Absolute Percentage Error (MDAPE)
    MDAPE = np.median(np.abs(errors) / (np.abs(y_true) + epsilon)) * 100
    
    return {
        'MAE': np.round(MAE, 2),
        'MedAE': np.round(MEDAE, 2),
        'MSE': np.round(MSE, 2),
        'RMSE': np.round(RMSE, 2),
        'MAPE': np.round(MAPE, 2),
        'MDAPE': np.round(MDAPE, 2)
    }

# Calculate and print metrics
metrics = calculate_metrics(y_test_unscaled, y_pred)

print("Evaluation Metrics:")
for name, value in metrics.items():
    if name in ['MAPE', 'MDAPE']:
        print(f"{name}: {value} %")
    else:
        print(f"{name}: {value}")

# Print shapes
print('\nShape Information:')
print(f'y_test_unscaled.shape = {y_test_unscaled.shape}')
print(f'y_pred.shape = {y_pred.shape}')

379/379 [==============================] - 1s 2ms/step
Evaluation Metrics:
MAE: 168.14
MedAE: 137.51
MSE: 47130.36
RMSE: 217.1
MAPE: 1.15 %
MDAPE: 0.95 %

Shape Information:
y_test_unscaled.shape = (12105, 1)
y_pred.shape = (12105, 1)


In [26]:
print('abc')

abc


# Fine Tuning

In [27]:
checkpoints = r'D:\anaconda\envs\lab7\checkpoint\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'D:\anaconda\envs\lab7\checkpoint\E1-cp-0059-loss0.01.h5'
start_epoch= 39

In [28]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading D:\anaconda\envs\lab7\checkpoint\E1-cp-0059-loss0.01.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [29]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2636/2653 [============================>.] - ETA: 0s - loss: 0.0089 - mae: 0.0089 - mape: 347.2217
Epoch 1: val_loss improved from inf to 0.00946, saving model to D:\anaconda\envs\lab7\checkpoint\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 9s 3ms/step - loss: 0.0089 - mae: 0.0089 - mape: 345.0714 - val_loss: 0.0095 - val_mae: 0.0095 - val_mape: 4.8290
Epoch 2/10
2632/2653 [============================>.] - ETA: 0s - loss: 0.0089 - mae: 0.0089 - mape: 332.0171
Epoch 2: val_loss improved from 0.00946 to 0.00913, saving model to D:\anaconda\envs\lab7\checkpoint\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 9s 3ms/step - loss: 0.0089 - mae: 0.0089 - mape: 329.4673 - val_loss: 0.0091 - val_mae: 0.0091 - val_mape: 4.6714
Epoch 3/10
2638/2653 [============================>.] - ETA: 0s - loss: 0.0089 - mae: 0.0089 - mape: 307.4911
Epoch 3: val_loss did not improve from 0.00913
2653/2653 [==============================] - 6s 2ms/step - lo

In [30]:

model = load_model(r'D:\anaconda\envs\lab7\checkpoint\E2-cp-0009-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 2ms/step
Mean Absolute Error (MAE): 146.27
Median Absolute Error (MedAE): 117.85
Mean Squared Error (MSE): 36411.63
Root Mean Squared Error (RMSE): 190.82
Mean Absolute Percentage Error (MAPE): 1.0 %
Median Absolute Percentage Error (MDAPE): 0.81 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
